In [1]:
%matplotlib inline

import datetime


In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
SOS_token = 0
EOS_token = 1

# 0, 1 are for "SOS" and "EOS"
# word id starts from 2
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.




In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.strip())
#     s = unicodeToAscii(s.lower().strip())
#     s = re.sub(r"([.!?])", r" \1", s)
#     s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the ``reverse``
flag to reverse the pairs.




In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
#     lines = open('headline_text.txt', encoding='utf-8').\
#         read().strip().split('\n')
    lines = open('eng-fra.txt', encoding='utf-8').\
        read().strip().split('\n')
    
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    print("Done reading")
    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 7

eng_prefixes = (
    "I am ", "I'm ",
    "He is", "He's ",
    "She is", "She's",
    "You are", "You're ",
    "We are", "We're ",
    "They are", "They're "
)

# def filterPair(p):
#     return True

# def filterPair(p):
#     return len(p[0].split(' ')) < MAX_LENGTH and \
#         len(p[1].split(' ')) < MAX_LENGTH 

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
Done reading
Read 160872 sentence pairs
Trimmed to 9438 sentence pairs
Counting words...
Counted words:
fra 5195
eng 3182
["Ton nom m'est familier.", 'I am familiar with your name.']


In [9]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input.cpu()).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output.cpu(), hidden.cpu())
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
# sentence to indices
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

# each string sentence is converted to integer strings
# and end with "EOS_TOKEN"
# note: it does not start with SOS_TOKEN
def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# tuple of two tensors
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [11]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    # Now: input_tensor and target_tensor are integer strings ending with <EOS>
    
    # obtain the initial hidden layer for encoder
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # number of tokens of the single sentence
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    loss = 0

    # loop through each token in a single input sentence
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

    # decoder takes a single token as input 
    decoder_input = torch.tensor([[SOS_token]], device=device)

    # pass the final hidden state of encoder
    decoder_hidden = encoder_hidden


    # loop over each token of the target sentence
    # decoder_input: single token
    for di in range(target_length):
        # decoder_output is a softmax vector over vocabulary size
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        topv, topi = decoder_output.topk(1) 
        
        # squeeze to get rid of dimensions. decoder_input is a scalar tensor
        # it is important not to use the SOS, EOS in the language model
        # because topi is the location of the most likely token
        decoder_input = topi.squeeze().detach()  # detach from history as input

        # compare the loss of the current predicted token (decoder_output)
        # and the ground truth token
        loss += criterion(decoder_output.cpu(), target_tensor[di].cpu())
        
        # if the most likely token is EOS, stop
        if decoder_input.item() == EOS_token:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length


In [12]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
#     encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
#     decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)

    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()
    best_encoder = []
    best_decoder = []
    lowest_loss = float("inf")
    for iter in range(1, n_iters + 1):
        # select 1 pair -> input, output tensors
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        # train based on 1 pair of input and output tensor
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        if loss < lowest_loss: 
            lowest_loss = loss
            best_encoder = encoder.state_dict()
            best_decoder = decoder.state_dict()
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)
    return encoder, decoder, encoder_optimizer, decoder_optimizer, loss

In [14]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [15]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # single sentence as a tensor
        input_tensor = tensorFromSentence(input_lang, sentence)
        
        # num tokens
        input_length = input_tensor.size()[0]
        
        encoder_hidden = encoder.initHidden()

        # loop over each token of the input sequence
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        # use SOS as the decoder input token
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        # feed the final hidden layer of encoder to the decoder
        decoder_hidden = encoder_hidden

        decoded_words = []

        # max length to sample
        for di in range(max_length):
            # Note: decoder_input is SOS at first
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            
            # pick the most likely word
            topv, topi = decoder_output.data.topk(1)
            
            # if it is EOS then, stop. But add <EOS> to it. 
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                # else, store the actual token as output. 
                decoded_words.append(output_lang.index2word[topi.item()])
            
            # squeeze the most likely token to a scalar for the next round
            decoder_input = topi.squeeze().detach()

        return decoded_words


In [16]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [17]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words)

In [22]:
encoder, decoder, encoder_optimizer, decoder_optimizer, loss = \
            trainIters(encoder1, decoder1, n_iters=1000, print_every=100)

0m 1s (- 0m 14s) (100 10%) 1.6168
0m 3s (- 0m 12s) (200 20%) 1.4469
0m 4s (- 0m 10s) (300 30%) 1.7727
0m 6s (- 0m 9s) (400 40%) 1.5333
0m 7s (- 0m 7s) (500 50%) 1.6930
0m 9s (- 0m 6s) (600 60%) 1.3627
0m 10s (- 0m 4s) (700 70%) 1.3902
0m 12s (- 0m 3s) (800 80%) 1.6049
0m 13s (- 0m 1s) (900 90%) 1.7134
0m 15s (- 0m 0s) (1000 100%) 1.5615


In [23]:
evaluateRandomly(encoder1, decoder1)

> Nous sommes heureux.
= We're happy.
< We're happy. happy. <EOS>

> Je n'en suis pas sur.
= I'm not sure about that.
< I'm not sure. sure. <EOS>

> Nous serons fermes demain.
= We're closed tomorrow.
< We're leaving tomorrow. tomorrow. tomorrow. <EOS>

> Vous etes suffisante.
= You're conceited.
< You're conceited. <EOS>

> Je suis simplement franche.
= I'm just being honest.
< I'm just just <EOS>

> J'en suis reellement fiere.
= I'm really proud of it.
< I'm really proud to that. that. <EOS>

> Je suis a court d'idees.
= I'm running out of ideas.
< I'm out out of <EOS>

> Vous n'etes pas chanteuse.
= You're no singer.
< You're no singer. <EOS>

> Tu n'es pas contusionne.
= You're not bruised.
< You're not bruised. <EOS>

> Je suis trop occupe pour l'aider.
= I'm too busy to help him.
< I'm too old to help <EOS>



In [20]:
def save_checkpoint(encoder, decoder, encoder_optimizer, decoder_optimizer,  loss, name="eng_fra_model.pt"):
    path = "./save/" + name
    torch.save({
                'encoder_model_state_dict': encoder.state_dict(),
                'decoder_model_state_dict': decoder.state_dict(),
                'encoder_optimizer_state_dict': encoder_optimizer.state_dict(),
                'decoder_optimizer_state_dict': decoder_optimizer.state_dict(),
                'timestamp': str(datetime.datetime.now()),
                'loss': loss,
                }, path)
save_checkpoint(encoder1, decoder1, encoder_optimizer, decoder_optimizer, loss)